# Error in 3D

In this notebook we load the points OpenPTV creates errors in 3D from 
Robin generated 2D and 3D points, see also proPTV_LineOfSight/generate_error.py

and markers_c0.txt, ...


In [1]:
import pandas as pd
import numpy as np
from typing import List, Tuple

from openptv_python.calibration import Calibration
from openptv_python.parameters import ControlPar, VolumePar, MultimediaPar
from openptv_python.orientation import skew_midpoint
from openptv_python.ray_tracing import ray_tracing

import pathlib, os
working_path = pathlib.Path.cwd()

In [2]:
class Parameter:
    cams = [0,1,2,3]
    Vmin = [0,0,0]
    Vmax = [300,300,300]
    N1, N2 = 361, 5

In [3]:
params = Parameter()

markers = [np.loadtxt('../proPTV_LineOfSight/markers_c'+str(cam)+'.txt') for cam in params.cams]
XYZ = markers[0][:,2:]
xy = markers[0][:,:2]
ID = np.argwhere((XYZ[:,0]>-1))[:,0]

In [4]:
xyz = pd.DataFrame(XYZ, columns=['x','y','z'])
xyz['id'] = ID
# px.scatter_3d(x=xyz['x'], y=xyz['y'], z=xyz['z'], color=xyz['id']).show()

In [5]:

cal = [Calibration().from_file(working_path / "calibration" / f"cam{cam_id+1}.tif.ori", None) for cam_id in range(4)]
# cal = [Calibration().from_file(working_path / "calibration" / f"cam{cam_id+1}.ori", working_path / "calibration" / f"cam{cam_id+1}.addpar") for cam_id in range(4)]
cpar = ControlPar().from_file(working_path / "parameters" / "ptv.par")
vpar = VolumePar().from_file(working_path / "parameters" / "criteria.par")

No addpar file found. Using default values for radial distortion
No addpar file found. Using default values for radial distortion
No addpar file found. Using default values for radial distortion
No addpar file found. Using default values for radial distortion


In [6]:
def point_position(
    targets: np.ndarray, # markers
    num_cams: int,
    multimed_pars: MultimediaPar,
    cals: List[Calibration],
) -> Tuple[float, np.ndarray]:
    """
    Calculate an average 3D position implied by the rays.

    sent toward it from cameras through the image projections of the point.

    Arguments:
    ---------
    targets - for each camera, the 2D metric, flat, centred coordinates
        of the identified point projection.
    multimed_pars - multimedia parameters struct for ray tracing through
        several layers.
    cals - each camera's calibration object.

    Returns
    -------
    A tuple containing the ray convergence measure (an average of skew ray distance across all ray pairs)
    and the average 3D position vector.
    """
    # loop counters
    num_used_pairs = 0
    dtot = 0.0
    point_tot = np.array([0.0, 0.0, 0.0])

    vertices = np.zeros((num_cams, 3))
    directs = np.zeros((num_cams, 3))
    point = np.zeros(3)
    
        # Shoot rays from all cameras.
    for cam in range(num_cams):
        vertices[cam], directs[cam] = ray_tracing(
            targets[cam][0], targets[cam][1], cals[cam], multimed_pars
        )

    # Check intersection distance for each pair of rays and find position
    for cam in range(num_cams):
        # if targets[cam][0] == COORD_UNUSED:
        #     continue

        for pair in range(cam + 1, num_cams):
            # if targets[pair][0] == COORD_UNUSED:
            #     continue

            num_used_pairs += 1
            tmp, point = skew_midpoint(
                vertices[cam], directs[cam], vertices[pair], directs[pair]
            )
            dtot += tmp
            point_tot += point

    res = point_tot / num_used_pairs
    dtot /= num_used_pairs

    return float(dtot), res.astype(float)

In [7]:
tmp = [[markers[cam][row,:2] for cam in range(4)] for row in range(1805)]

In [8]:
for row in tmp:
    eps, pos = point_position(row, 4, cpar.mm, cal)
    print(eps, pos)
    if row == 10:
        break

4749.196305358981 [  5280.2220318   31898.37884946 -30035.00501916]
4793.708161649144 [-2974.87273273 -2565.04410069  2193.70767767]
4708.562841363447 [-11846.07080917 -27796.69458989  26459.85818839]
4755.268568575087 [-13251.70265758 -27113.59604052  26261.02715149]
4966.557854793726 [-11919.53029892 -20350.61458719  20033.10982964]
5521.177472378587 [ -9397.0210205  -12582.43891991  12509.68586229]
6427.804502261369 [-6217.97719778 -4938.17353141  4732.89880014]
7630.04109194793 [-2360.73854688  2742.10982737 -3546.27269145]
9300.165628255012 [  3352.15868281  12404.4399006  -14600.35170072]
11892.734363697817 [ 12995.97782399  26933.08056615 -32083.79687532]
16806.222277346864 [ 33975.90129708  55994.31064674 -68305.75665549]
30561.24133492364 [ 102674.54107158  146273.66900857 -183417.77041057]
1027335.8933174075 [ 5679383.71834264  7255262.32701538 -9362773.87312251]
29915.481778279544 [-219101.7894134  -255722.4763855   339683.82336741]
15997.261916323268 [-155708.09361095 -1681